In [1]:
# 修改预训练模型的缓存目录
import os

os.environ["MODELSCOPE_CACHE"] = r"G:\code\pretrain_model_dir\_modelscope"

In [2]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2-VL-7B-Instruct')

2024-10-24 22:06:26,437 - modelscope - INFO - PyTorch version 2.1.0+cu121 Found.
2024-10-24 22:06:26,440 - modelscope - INFO - TensorFlow version 2.8.0 Found.
2024-10-24 22:06:26,440 - modelscope - INFO - Loading ast index from G:\code\pretrain_model_dir\_modelscope\ast_indexer
2024-10-24 22:06:26,806 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 ba00b7358e147b49b70e6555c5c8ccf2 and a total number of 964 components indexed
c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.03k/1.03k [00:00<00:00, 246kB/s]
Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 206kB/s]
Downloading: 100%|██████████| 76.0/76.0 [00:00<00:00, 11.9kB/s]
Downloading: 100%|██████████| 244/244 [00:00<00:00, 43.8kB/s]
Downloading: 100%|██████████| 

In [3]:
print(model_dir)

G:\code\pretrain_model_dir\_modelscope\qwen\Qwen2-VL-7B-Instruct


In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from modelscope import snapshot_download
import torch
model_dir = r"G:\code\pretrain_model_dir\_modelscope\qwen\Qwen2-VL-7B-Instruct"

# default: Load the model on the available device(s)
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_dir, torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [00:30<00:00,  6.16s/it]


In [4]:
print(model.device, model.dtype)

cuda:0 torch.bfloat16


In [2]:
model

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Sequent

In [5]:

# default processer
processor = AutoProcessor.from_pretrained(model_dir)

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(model_dir, min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")



In [6]:
print(inputs.keys())

for key, val in inputs.items():
    print(key, val.shape)
print(inputs)

dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
input_ids torch.Size([1, 1272])
attention_mask torch.Size([1, 1272])
pixel_values torch.Size([4988, 1176])
image_grid_thw torch.Size([1, 3])
{'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[ 0.8501,  0.8647,  0.8647,  ...,  1.4207,  1.4349,  1.4349],
        [ 1.0106,  1.0106,  1.0252,  ...,  1.5060,  1.5060,  1.5202],
        [ 0.9960,  1.0106,  1.0252,  ...,  1.5060,  1.5202,  1.5202],
        ...,
        [-0.2448, -0.2594, -0.2886,  ..., -0.1862, -0.1720, -0.1862],
        [-0.1426, -0.1134, -0.0550,  ..., -0.1293, -0.1435, -0.1720],
        [-0.2448, -0.2448, -0.2594,  ..., -0.3853, -0.4279, -0.4564]],
       device='cuda:0'), 'image_grid_thw': tensor([[ 1, 58, 86]], device='cuda:0', dtype=torch.int32)}


In [7]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

["The image depicts a serene beach scene at sunset. A woman and her dog are sitting on the sand, enjoying each other's company. The woman is wearing a plaid shirt and dark pants, and she is smiling as she reaches out to her dog. The dog, which appears to be a large breed, is wearing a harness and is giving a high-five to the woman. The beach is relatively empty, with gentle waves in the background and a clear sky. The warm, golden light of the setting sun casts a soft glow over the entire scene, creating a peaceful and joyful atmosphere."]
